In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
standings_urls = ["https://fbref.com/en/comps/9/Premier-League-Stats",
    "https://fbref.com/en/comps/11/Serie-A-Stats",
    "https://fbref.com/en/comps/12/La-Liga-Stats",
    "https://fbref.com/en/comps/13/Ligue-1-Stats",
    "https://fbref.com/en/comps/20/Bundesliga-Stats"]

leagues =  pd.read_excel('./../Club_name.xlsx')

In [3]:
leagues.head()

,EPL,Serie A,La Liga,Ligue 1,Bundesliga
0,Arsenal,Napoli,Barcelona,Paris S-G,Union Berlin
1,Manchester City,Inter,Real Madrid,Marseille,Bayern Munich
2,Manchester Utd,Roma,Real Sociedad,Lens,Dortmund
3,Newcastle Utd,Lazio,AtlÃ©tico Madrid,Monaco,RB Leipzig
4,Tottenham,Atalanta,Villarreal,Rennes,Eint Frankfurt


In [4]:
def clean(table, league):
    if ( 'Unnamed: 4_level_0',       'MP') in table.columns:    
        table = table[[( 'Unnamed: 0_level_0',   'Player'),
            ( 'Unnamed: 1_level_0',   'Nation'),
            ( 'Unnamed: 2_level_0',      'Pos'),
            ( 'Unnamed: 3_level_0',      'Age'),
            ( 'Unnamed: 4_level_0',       'MP'),
            (        'Performance',      'Gls'),
            (        'Performance',      'Ast')]][:-2]
    else:
        table = table[[( 'Unnamed: 0_level_0',   'Player'),
            ( 'Unnamed: 1_level_0',   'Nation'),
            ( 'Unnamed: 2_level_0',      'Pos'),
            ( 'Unnamed: 3_level_0',      'Age'),
            ( 'Playing Time',       'MP'),
            (        'Performance',      'Gls'),
            (        'Performance',      'Ast')]][:-2]

    table.columns = table.columns.droplevel()
    table['Gls'] = table['Gls'].fillna(0).astype(int)
    table['Ast'] = table['Ast'].fillna(0).astype(int)
    table['Nation'] = [m[-3:] if (type(m) == str and len(m) >= 3) else None for m in table['Nation']]
    table['Age'] = [m[:2] if (type(m) == str and len(m) > 0) else None for m in table['Age']]
    return table

In [13]:
player_df = pd.DataFrame(columns=['Player', "Nation", 'Pos', 'Age', "MP", 'Gls', 'Ast'])
count = 0

In [16]:
player_df = pd.read_csv('./playerDB60.csv')
player_df.describe()

,Age,MP,Gls,Ast
count,2041.000000,2042.000000,2042.000000,2042.000000
mean,25.499755,9.295788,0.753673,0.532321
std,4.771147,7.582030,1.757618,1.104169
min,15.000000,0.000000,0.000000,0.000000
25%,22.000000,1.000000,0.000000,0.000000
50%,25.000000,10.000000,0.000000,0.000000
75%,29.000000,17.000000,1.000000,1.000000
max,41.000000,22.000000,25.000000,11.000000


In [24]:
print(leagues[['Ligue 1', 'Bundesliga']])

          Ligue 1      Bundesliga
0       Paris S-G    Union Berlin
1       Marseille   Bayern Munich
2            Lens        Dortmund
3          Monaco      RB Leipzig
4          Rennes  Eint Frankfurt
5           Lille        Freiburg
6         Lorient       Wolfsburg
7            Lyon      M'Gladbach
8            Nice      Leverkusen
9   Clermont Foot   Werder Bremen
10          Reims           KÃ¶ln
11       Toulouse        Mainz 05
12         Nantes        Augsburg
13    Montpellier      Hoffenheim
14          Brest          Bochum
15         Troyes       Stuttgart
16        Ajaccio      Hertha BSC
17     Strasbourg      Schalke 04
18        Auxerre             NaN
19         Angers             NaN


In [25]:
for standings_url, league in zip(standings_urls[3:],leagues[['Ligue 1', 'Bundesliga']]):
    data = requests.get(standings_url) 
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]
    links = standings_table.find_all('a')
    links = [l.get("href") for l in links]
    links = [l for l in links if '/squads/' in l]
    standing = pd.read_html(data.text, attrs={'class': 'stats_table'})[0]

    team_urls = [f"https://fbref.com{l}" for l in links]

    for i, (l, team_name) in enumerate((zip(team_urls, leagues[league]))):
        if i == len(team_urls): 
            break     
        d = requests.get(l)
        match = clean(pd.read_html(d.text, match = "Standard Stats")[0], league)
        player_df = pd.concat([player_df, match])
        print(count)
        count += 1
    player_df.to_csv(f'./playerDB{count}.csv', index=False)

60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97


In [26]:
player_df.describe()

,MP,Gls,Ast
count,3273.000000,3273.000000,3273.000000
mean,9.468989,0.822487,0.577146
std,7.546132,1.831024,1.180885
min,0.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000
50%,10.000000,0.000000,0.000000
75%,17.000000,1.000000,1.000000
max,22.000000,25.000000,11.000000


In [27]:
player_info_df = player_df[['Player', 'Nation', 'Pos', 'Age']]

In [31]:
player_info_df.head()

,player_id,Player,Nation,Pos,Age
0,1,Gabriel Dos Santos,BRA,DF,25.0
1,2,Aaron Ramsdale,ENG,GK,24.0
2,3,William Saliba,FRA,DF,21.0
3,4,Bukayo Saka,ENG,FW,21.0
4,5,Granit Xhaka,SUI,MF,30.0


In [30]:
player_info_df.insert(0, 'player_id', list(range(1, len(player_info_df.index) + 1)), False)

In [32]:
player_info_df.describe()

,player_id
count,3273.000000
mean,1637.000000
std,944.978042
min,1.000000
25%,819.000000
50%,1637.000000
75%,2455.000000
max,3273.000000


In [34]:
is_member_df = player_df[['MP', 'Gls', 'Ast']]
is_member_df.head()

,MP,Gls,Ast
0,20,2,0
1,20,0,0
2,20,2,1
3,20,7,7
4,20,3,5


In [35]:
is_member_df.insert(0, 'player_id', list(range(1, len(is_member_df.index) + 1)), False)
is_member_df.head()

,player_id,MP,Gls,Ast
0,1,20,2,0
1,2,20,0,0
2,3,20,2,1
3,4,20,7,7
4,5,20,3,5


In [36]:
is_member_df.insert(1, 'season', '2022-2023', True)
is_member_df.head()

,player_id,season,MP,Gls,Ast
0,1,2022-2023,20,2,0
1,2,2022-2023,20,0,0
2,3,2022-2023,20,2,1
3,4,2022-2023,20,7,7
4,5,2022-2023,20,3,5
